### Version 1 Basic

In [3]:
import streamlit as st
import pandas as pd
import joblib

# Load the model pipeline
model_pipeline = joblib.load('lasso_pipeline.joblib')

# Load the master data
master_data_path = 'master_data.xlsx'
master_data = pd.read_excel(master_data_path)

# List of selected columns, make sure to define this correctly

selected_columns = ['Province', 'Paralegals and Social Service Workers', 'Inflation Rate', 'Mortgage Rate', 
                    'Temp_Resident_to_Permanent_Resident', 'Population', 'Resettled_Refugee', 'Worker_Program',
                    'Construction and Equipment Operations', 'Vacancy Rate', 'COVID_era', 'Business_Immigration',
                    'Construction and Facility Management', 'Transportation and Natural Resources Management']

# Function to accept user input

def user_input_features():
    st.sidebar.header('Enter the feature values:')
    
    # Categorical feature input
    province = st.sidebar.selectbox('Province', master_data['Province'].unique())
    
    # Numeric features input
    inputs = {
        'Year': st.sidebar.number_input('Year', min_value=int(master_data['Year'].min()), max_value=int(master_data['Year'].max()), value=int(master_data['Year'].min())),
    }

    # Collect numeric inputs in a loop
    for feature in selected_columns[1:]:  # Excluding 'Province' which we already added
        inputs[feature] = st.sidebar.number_input(feature, value=0)  # You can set a default value
    
    # Create a DataFrame from the inputs
    features_df = pd.DataFrame([inputs])
    features_df['Province'] = province  # Add the categorical feature
    return features_df

def main():
    # Application title
    st.title('HPI Prediction using Lasso Regression')
    
    # Get user input features from the sidebar
    input_df = user_input_features()
    
    # Predict button
    if st.button('Predict HPI'):
        # Make predictions
        prediction = model_pipeline.predict(input_df)
        
        # Output the prediction
        st.write(f'Predicted HPI: {prediction[0]}')

# Run the main function
if __name__ == '__main__':
    main()


### Version 2 adding tabs

In [ ]:
import streamlit as st
import pandas as pd
import joblib

# Load the model pipeline
model_pipeline = joblib.load('lasso_pipeline.joblib')

# Load the master data
master_data_path = 'master_data.xlsx'
master_data = pd.read_excel(master_data_path)

# Different views for different users
policy_features = ['Province', 'Paralegals and Social Service Workers', 'Inflation Rate', 'Mortgage Rate', 
                   'Temp_Resident_to_Permanent_Resident', 'Population', 'Resettled_Refugee', 'Worker_Program',
                   'Construction and Equipment Operations', 'Vacancy Rate', 'COVID_era', 'Business_Immigration',
                   'Construction and Facility Management', 'Transportation and Natural Resources Management']

investor_features = ['Inflation Rate', 'Mortgage Rate', 'Vacancy Rate']

def user_input_features(features):
    st.sidebar.header('Enter the feature values:')
    inputs = {}
    # Categorical feature input
    if 'Province' in features:
        inputs['Province'] = st.sidebar.selectbox('Province', master_data['Province'].unique())
    
    # Numeric features input
    for feature in features:
        if feature != 'Province':  # Exclude Province from the numeric inputs
            inputs[feature] = st.sidebar.number_input(feature, value=0)
    
    # Create a DataFrame from the inputs
    features_df = pd.DataFrame([inputs])
    if 'Province' in inputs:
        features_df['Province'] = inputs['Province']  # Add the categorical feature
    return features_df

def main():
    # Application title
    st.title('HPI Prediction Tool')
    
    # Custom CSS to inject into the webpage to display the background image
    st.markdown(
    f"""
    <style>
    .stApp {{
        background-image: url("https://img.freeflagicons.com/thumb/house_with_flag/canada/canada_640.png");
        background-size: cover;
        background-repeat: no-repeat;
        background-position: center;
    }}
    </style>
    """,
    unsafe_allow_html=True
)

    
    tab1, tab2 = st.tabs(["Policy Maker View", "Investor View"])
    
    with tab1:
        st.write("Policy Maker Dashboard")
        input_df = user_input_features(policy_features)
        if st.button('Predict HPI for Policymakers'):
            prediction = model_pipeline.predict(input_df)
            st.write(f'Predicted HPI: {prediction[0]}')

    with tab2:
        st.write("Real Estate Investor Dashboard")
        input_df = user_input_features(investor_features)
        if st.button('Predict HPI for Investors'):
            prediction = model_pipeline.predict(input_df)
            st.write(f'Predicted HPI: {prediction[0]}')

if __name__ == '__main__':
    main()


In [ ]:
def user_input_features(features):
    st.sidebar.header('Enter the feature values:')
    
    inputs = {}
    # Categorical feature input
    if 'Province' in features:
        inputs['Province'] = st.sidebar.selectbox('Province', master_data['Province'].unique(), key='Province_selectbox')
    
    # Numeric features input
    for feature in features:
        if feature != 'Province':  # Exclude Province from the numeric inputs
            # Generate a unique key for each input using the feature name
            key = f"input_{feature.replace(' ', '_').lower()}"  # Create a unique key for each feature
            inputs[feature] = st.sidebar.number_input(feature, value=0, key=key)
    
    # Create a DataFrame from the inputs
    features_df = pd.DataFrame([inputs])
    if 'Province' in inputs:
        features_df['Province'] = inputs['Province']  # Add the categorical feature
    return features_df

### Version 3 fixing the key error

In [ ]:
import streamlit as st
import pandas as pd
import joblib

# Load the model pipeline
model_pipeline = joblib.load('lasso_pipeline.joblib')

# Load the master data
master_data_path = 'master_data.xlsx'
master_data = pd.read_excel(master_data_path)

# Different views for different users
policy_features = ['Province', 'Paralegals and Social Service Workers', 'Inflation Rate', 'Mortgage Rate', 
                   'Temp_Resident_to_Permanent_Resident', 'Population', 'Resettled_Refugee', 'Worker_Program',
                   'Construction and Equipment Operations', 'Vacancy Rate', 'COVID_era', 'Business_Immigration',
                   'Construction and Facility Management', 'Transportation and Natural Resources Management']

investor_features = ['Inflation Rate', 'Mortgage Rate', 'Vacancy Rate']

def user_input_features(features, context):
    st.sidebar.header(f'Enter the feature values for {context}:')
    
    inputs = {}
    # Categorical feature input
    if 'Province' in features:
        inputs['Province'] = st.sidebar.selectbox('Province', master_data['Province'].unique(), key=f'{context}_Province_selectbox')
    
    # Numeric features input
    for feature in features:
        if feature != 'Province':  # Exclude Province from the numeric inputs
            # Generate a unique key for each input using the feature name and context
            key = f"{context}_{feature.replace(' ', '_').lower()}"  # Create a unique key for each feature
            inputs[feature] = st.sidebar.number_input(feature, value=0, key=key)
    
    # Create a DataFrame from the inputs
    features_df = pd.DataFrame([inputs])
    if 'Province' in inputs:
        features_df['Province'] = inputs['Province']  # Add the categorical feature
    return features_df

def main():
    # Application title
    st.title('HPI Prediction Tool')
    
    # Custom CSS to inject into the webpage to display the background image
    st.markdown(
    f"""
    <style>
    .stApp {{
        background-image: url("https://img.freeflagicons.com/thumb/house_with_flag/canada/canada_640.png");
        background-size: cover;
        background-repeat: no-repeat;
        background-position: center;
    }}
    </style>
    """,
    unsafe_allow_html=True
)

    
    tab1, tab2 = st.tabs(["Policy Maker View", "Investor View"])
    
    with tab1:
        st.write("Policy Maker Dashboard")
        input_df = user_input_features(policy_features, 'policymaker')
        if st.button('Predict HPI for Policymakers', key='predict_policymaker'):
            prediction = model_pipeline.predict(input_df)
            st.write(f'Predicted HPI: {prediction[0]}')

    with tab2:
        st.write("Real Estate Investor Dashboard")
        input_df = user_input_features(investor_features, 'investor')
        if st.button('Predict HPI for Investors', key='predict_investor'):
            prediction = model_pipeline.predict(input_df)
            st.write(f'Predicted HPI: {prediction[0]}')

if __name__ == '__main__':
    main()


### Version 4 adding images and changing the sidebar view

In [ ]:
import streamlit as st
import pandas as pd
import joblib

# Load the model pipeline
model_pipeline = joblib.load('lasso_pipeline.joblib')

# Load the master data
master_data_path = 'master_data.xlsx'
master_data = pd.read_excel(master_data_path)

# Different views for different users
policy_features = [
    'Province', 'Paralegals and Social Service Workers', 'Inflation Rate', 'Mortgage Rate',
    'Temp_Resident_to_Permanent_Resident', 'Population', 'Resettled_Refugee', 'Worker_Program',
    'Construction and Equipment Operations', 'Vacancy Rate', 'COVID_era', 'Business_Immigration',
    'Construction and Facility Management', 'Transportation and Natural Resources Management'
]

investor_features = ['Inflation Rate', 'Mortgage Rate', 'Vacancy Rate']

def user_input_features(features, context):
    inputs = {}
    # Categorical feature input
    if 'Province' in features:
        inputs['Province'] = st.sidebar.selectbox('Province', master_data['Province'].unique(), key=f'{context}_Province_selectbox')
    
    # Numeric features input
    for feature in features:
        if feature != 'Province':  # Exclude Province from the numeric inputs
            # Generate a unique key for each input using the feature name and context
            key = f"{context}_{feature.replace(' ', '_').lower()}"  # Create a unique key for each feature
            inputs[feature] = st.sidebar.number_input(feature, value=0, key=key)
    
    return pd.DataFrame([inputs])

def main():
    st.title('HPI Prediction Tool')

    # Custom CSS to inject into the webpage to display the background image
    st.markdown(
        """
        <style>
        .stApp {
            background-image: url("https://img.freeflagicons.com/thumb/house_with_flag/canada/canada_640.png");
            background-size: cover;
            background-repeat: no-repeat;
            background-position: center;
        }
        </style>
        """,
        unsafe_allow_html=True
    )

    tab1, tab2 = st.tabs(["Policy Maker View", "Investor View"])

    with tab1:
        st.write("Policy Maker Dashboard")
        if st.button('Show inputs for Policymakers', key='show_policymaker'):
            input_df = user_input_features(policy_features, 'policymaker')
        if st.button('Predict HPI for Policymakers', key='predict_policymaker'):
            prediction = model_pipeline.predict(input_df)
            st.write(f'Predicted HPI: {prediction[0]}')

    with tab2:
        st.write("Real Estate Investor Dashboard")
        if st.button('Show inputs for Investors', key='show_investor'):
            input_df = user_input_features(investor_features, 'investor')
        if st.button('Predict HPI for Investors', key='predict_investor'):
            prediction = model_pipeline.predict(input_df)
            st.write(f'Predicted HPI: {prediction[0]}')

if __name__ == '__main__':
    main()


# Version 5 fixing the error of entering values

In [ ]:
import streamlit as st
import pandas as pd
import joblib

# Load the model pipeline
model_pipeline = joblib.load('lasso_pipeline.joblib')

# Load the master data
master_data_path = 'master_data.xlsx'
master_data = pd.read_excel(master_data_path)

# Different views for different users
policy_features = ['Province', 'Paralegals and Social Service Workers', 'Inflation Rate', 'Mortgage Rate', 
                   'Temp_Resident_to_Permanent_Resident', 'Population', 'Resettled_Refugee', 'Worker_Program',
                   'Construction and Equipment Operations', 'Vacancy Rate', 'COVID_era', 'Business_Immigration',
                   'Construction and Facility Management', 'Transportation and Natural Resources Management']

investor_features = ['Inflation Rate', 'Mortgage Rate', 'Vacancy Rate']

def user_input_features(features, context):
    inputs = {}
    # Categorical feature input
    if 'Province' in features:
        inputs['Province'] = st.sidebar.selectbox('Province', master_data['Province'].unique(), key=f'{context}_Province_selectbox')
    
    # Numeric features input
    for feature in features:
        if feature != 'Province':  # Exclude Province from the numeric inputs
            # Generate a unique key for each input using the feature name and context
            key = f"{context}_{feature.replace(' ', '_').lower()}"  # Create a unique key for each feature
            inputs[feature] = st.sidebar.number_input(feature, value=0, key=key)
    
    return pd.DataFrame([inputs])

def main():
    st.title('HPI Prediction Tool')

    # Custom CSS to inject into the webpage to display the background image
    st.markdown(
        """
        <style>
        .stApp {
            background-image: url("https://img.freeflagicons.com/thumb/house_with_flag/canada/canada_640.png");
            background-size: cover;
            background-repeat: no-repeat;
            background-position: center;
        }
        </style>
        """,
        unsafe_allow_html=True
    )

    tab1, tab2 = st.tabs(["Policy Maker View", "Investor View"])
    
    with tab1:
        st.write("Policy Maker Dashboard")
        input_df = None  # Initialize input_df to None
        if st.button('Show inputs for Policymakers', key='show_policymaker'):
            input_df = user_input_features(policy_features, 'policymaker')
        if st.button('Predict HPI for Policymakers', key='predict_policymaker') and input_df is not None:
            prediction = model_pipeline.predict(input_df)
            st.write(f'Predicted HPI: {prediction[0]}')

    with tab2:
        st.write("Real Estate Investor Dashboard")
        input_df = None  # Initialize input_df to None
        if st.button('Show inputs for Investors', key='show_investor'):
            input_df = user_input_features(investor_features, 'investor')
        if st.button('Predict HPI for Investors', key='predict_investor') and input_df is not None:
            prediction = model_pipeline.predict(input_df)
            st.write(f'Predicted HPI: {prediction[0]}')

if __name__ == '__main__':
    main()
